<a href="https://colab.research.google.com/github/IndraP24/MyCaptain-Codes/blob/master/Test%20Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#importing dependencies
import numpy as np
import sys
import nltk
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Using TensorFlow backend.


In [0]:
#load data
#loading data and opening our input data in the form of a txt file
#Project Gutenberg is where the data can be found
file = open("84-0.txt").read()

In [0]:
#tokenization
#standardization
def tokenize_words(input):
  #lowercase everything to standardize it
  input = input.lower()
  #instantiating the tokenizer
  tokenizer = RegexpTokenizer(r'\w+')
  #tokenizing the text into tokens
  tokens = tokenizer.tokenize(input)
  #filtering the stopwords using lambda
  filtered = filter(lambda token: token not in stopwords.words('english'), tokens)
  return "".join(filtered)

#preprocess the input data, make tokens
processed_inputs = tokenize_words(file)

In [0]:
#chars to numbers
chars = sorted(list(set(processed_inputs)))
char_to_num = dict((c, i) for i, c in enumerate(chars))

In [0]:
#check if words to chars or chars to num (?!) has worked
input_len = len(processed_inputs)
vocab_len = len(chars)
print('Total No. of Characters: ', input_len)
print('Total vocab: ', vocab_len)

Total No. of Characters:  233296
Total vocab:  42


In [0]:
#seq length
seq_length = 100
x_data = []
y_data = []

In [0]:
#loop through the sequence
for i in range(0, input_len - seq_length, 1):
  in_seq = processed_inputs[i:i + seq_length]
  out_seq = processed_inputs[i + seq_length]
  x_data.append([char_to_num[char] for char in in_seq])
  y_data.append(char_to_num[out_seq])

n_patterns = len(x_data)
print('Total Patterns:', n_patterns)

Total Patterns: 233196


In [0]:
#convert input sequence to np array that our network can use
x = np.reshape(x_data, (n_patterns, seq_length, 1))
x = x / float(vocab_len)

In [0]:
#one hot encoding our label data
y = np_utils.to_categorical(y_data)


In [0]:
#creating the sequential model
#dropout is uesd to predict
model = Sequential()
model.add(LSTM(256, input_shape = (x.shape[1], x.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

In [0]:
#compile the model
model.compile(loss = 'categorical_crossentropy', optimizer='adam')

In [0]:
#saving weights
filepath = 'model_weights_saved.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor = 'loss', verbose = 1, save_best_only=True, mode = 'min')
desired_callbacks = [checkpoint]

In [0]:
model.fit(x, y, epochs = 4, batch_size=256, callbacks=desired_callbacks)

Epoch 1/4
233196/233196 [==============================] - 569s 2ms/step - loss: 2.9310

Epoch 00001: loss improved from inf to 2.93103, saving model to model_weights_saved.hdf5
Epoch 2/4
233196/233196 [==============================] - 567s 2ms/step - loss: 2.9105

Epoch 00002: loss improved from 2.93103 to 2.91045, saving model to model_weights_saved.hdf5
Epoch 3/4
233196/233196 [==============================] - 565s 2ms/step - loss: 2.8680

Epoch 00003: loss improved from 2.91045 to 2.86804, saving model to model_weights_saved.hdf5
Epoch 4/4
233196/233196 [==============================] - 565s 2ms/step - loss: 2.8000

Epoch 00004: loss improved from 2.86804 to 2.79997, saving model to model_weights_saved.hdf5


In [0]:
# recompile model with the saved weights
filename = "model_weights_saved.hdf5"
model.load_weights(filename)
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

In [0]:
#output of the model back into characters
num_to_char = dict((i, c) for i, c in enumerate(chars))

In [0]:
#random seed to help generate
start = np.random.randint(0, len(x_data) - 1)
pattern = x_data[start]
print("Random Seed:")
print("\"", ''.join([num_to_char[value] for value in pattern]), "\"")

Random Seed:
" etseashoreinquiredinhabitantsconcerningfiendgainedaccurateinformationgiganticmonstersaidarrivednight "


In [0]:
#generate the text
for i in range(1000):
  x = np.reshape(pattern, (1, len(pattern), 1))
  x = x / float(vocab_len)
  prediction = model.predict(x, verbose = 0)
  index = np.argmax(prediction)
  result = num_to_char[index]
  seq_in = [num_to_char[value] for value in pattern]
  sys.stdout.write(result)
  pattern.append(index)
  pattern = pattern[1:len(pattern)]

ereeredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredredred